In [8]:
import tensorflow as tf
import sys
sys.path.append('/nas/longleaf/home/siyangj/NiftyNet/')
import niftynet as nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import SimpleITK as sitk
import os
import re
import nibabel as nib
import sklearn as skl
import pandas as pd

pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 10000

In [9]:
def Dice(pred,true,labels=[0,1,2,3]):
    l = len(labels)
    ret = np.zeros(l)
    for i in range(l):
        lab = labels[i]
        p = pred==lab
        t = true==lab
        ret[i] = 2*np.logical_and(p,t).sum()/(p.sum()+t.sum())
    return ret

In [10]:
def inference_eval(model,inf_dir,data_path,eval_df,test_im,
                   seg_suffix='seg/',skip_if_exist=True):
    inf_re = re.compile(r'inference.*')
    im_re = re.compile(r'.*\.nii\.gz')
    model_path = '/proj/NIRAL/users/siyangj/model_{}/'.format(model)
    
    inf_ims = {f[0]:os.path.join(inf_dir,f) for f in os.listdir(inf_dir) 
            if (os.path.isfile(os.path.join(inf_dir, f)) and im_re.match(f) is not None)}
    l = len(eval_df.index)
    if len(eval_df.loc[eval_df['inf_dir']==inf_dir])>0:
        if skip_if_exist:
            return
        l = np.where(eval_df['inf_dir']==inf_dir)[0][0]
    eval_df.loc[l,'inf_dir']=inf_dir
    eval_df.loc[l,'model']=model
    eval_df.loc[l,'test_im']=test_im
    for im_num,im in inf_ims.items():
        true_im = os.path.join(data_path,seg_suffix,im_num+'.nrrd')
        true_arr = sitk.GetArrayFromImage(sitk.ReadImage(true_im))
        #print(true_arr.shape)
        inf_arr = sitk.GetArrayFromImage(sitk.ReadImage(im))
        #print(inf_arr.shape)
        c_mat = skl.metrics.confusion_matrix(true_arr.flatten(),inf_arr.flatten())
        p_mat = c_mat.T / c_mat.astype(np.float).sum(axis=1)
        eval_df.loc[l,im_num]=[p_mat.T]
        if im_num == test_im:
            eval_df.loc[l,'test_11'] = p_mat[1,1]
            eval_df.loc[l,'test_12'] = p_mat[2,1]
            eval_df.loc[l,'test_21'] = p_mat[1,2]
            eval_df.loc[l,'test_22'] = p_mat[2,2]
            eval_df.loc[l,'test_11+22'] = eval_df.loc[l,'test_11']+eval_df.loc[l,'test_22']
            eval_df.loc[l,'test_12+21'] = eval_df.loc[l,'test_12']+eval_df.loc[l,'test_21']
            Dice0123 = Dice(inf_arr,true_arr)
            eval_df.loc[l,'Dice0'] = Dice0123[0]
            eval_df.loc[l,'Dice1'] = Dice0123[1]
            eval_df.loc[l,'Dice2'] = Dice0123[2]
            eval_df.loc[l,'Dice3'] = Dice0123[3]

In [4]:
def model_eval(model,data_path,eval_df,test_im,seg_suffix='seg/'):
    model_path = '/proj/NIRAL/users/siyangj/model_{}/'.format(model)
    inf_re = re.compile(r'inference.*')
    im_re = re.compile(r'.*\.nii\.gz')
    inf_dirs = [f for f in os.listdir(model_path) 
            if (os.path.isdir(os.path.join(model_path, f)) 
                and inf_re.match(f) is not None)]
    for d in inf_dirs:
        inf_dir = os.path.join(model_path,d)
        inference_eval(model,inf_dir,data_path,eval_df,test_im,seg_suffix=seg_suffix)

In [5]:
eval_df=pd.read_csv('/proj/NIRAL/users/siyangj/inference_eval_4.csv')

In [11]:
eval_df['model']=eval_df['model'].astype(int)

In [12]:
eval_df['test_im']=eval_df['test_im'].astype(int)

In [13]:
eval_df

,model,inf_dir,test_im,test_11,test_12,test_21,test_22,test_11+22,test_12+21,Dice0,Dice1,Dice2,Dice3,1,2,3,4,5,6,7,8
0,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference_17000_10112050,1,0.918915,0.067399,0.295044,0.649303,1.568218,0.362443,0.993768,0.780229,0.744064,0.805368,"[array([[9.99995958e-01, 0.00000000e+00, 0.00000000e+00, 4.04240483e-06],\n [6.64359862e-0...","[array([[9.92899343e-01, 2.40386838e-05, 2.32620494e-03, 4.75041374e-03],\n [1.10490523e-0...","[array([[9.99996610e-01, 0.00000000e+00, 0.00000000e+00, 3.38991095e-06],\n [0.00000000e+0...","[array([[9.99996543e-01, 0.00000000e+00, 0.00000000e+00, 3.45705559e-06],\n [0.00000000e+0...","[array([[9.99925734e-01, 0.00000000e+00, 1.90424685e-06, 7.23613804e-05],\n [0.00000000e+0...","[array([[9.99957277e-01, 0.00000000e+00, 0.00000000e+00, 4.27234118e-05],\n [0.00000000e+0...","[array([[9.99967062e-01, 0.00000000e+00, 0.00000000e+00, 3.29379214e-05],\n [0.00000000e+0...","[array([[9.99829622e-01, 0.00000000e+00, 7.67470310e-06, 1.62703706e-04],\n [0.00000000e+0..."
1,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference,1,0.895175,0.097146,0.224378,0.753510,1.648685,0.321524,0.991811,0.803432,0.796687,0.804782,"[array([[9.99987873e-01, 0.00000000e+00, 0.00000000e+00, 1.21272145e-05],\n [6.64359862e-0...","[array([[9.92847567e-01, 0.00000000e+00, 6.71788755e-03, 4.34545438e-04],\n [1.21539575e-0...","[array([[9.99969491e-01, 0.00000000e+00, 1.01697328e-05, 2.03394657e-05],\n [1.98668918e-0...","[array([[9.99970615e-01, 0.00000000e+00, 1.20996946e-05, 1.72852780e-05],\n [3.00979689e-0...","[array([[9.99849564e-01, 0.00000000e+00, 2.66594559e-05, 1.23776045e-04],\n [0.00000000e+0...","[array([[9.99917718e-01, 0.00000000e+00, 9.49409151e-06, 7.27880349e-05],\n [0.00000000e+0...","[array([[9.99855701e-01, 0.00000000e+00, 6.11704255e-05, 8.31290398e-05],\n [7.04528001e-0...","[array([[9.99889484e-01, 0.00000000e+00, 4.75831592e-05, 6.29325655e-05],\n [0.00000000e+0..."
2,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference_13200_10112050,1,0.926572,0.058502,0.308779,0.649881,1.576453,0.367281,0.994202,0.777533,0.752149,0.844704,"[array([[9.99981809e-01, 0.00000000e+00, 0.00000000e+00, 1.81908217e-05],\n [1.10726644e-0...","[array([[9.95715567e-01, 0.00000000e+00, 3.19899407e-03, 1.08543903e-03],\n [7.18188397e-0...","[array([[9.99976271e-01, 0.00000000e+00, 1.01697328e-05, 1.35596438e-05],\n [2.33728139e-0...","[array([[9.99987900e-01, 0.00000000e+00, 3.45705559e-06, 8.64263898e-06],\n [2.50816407e-0...","[array([[9.99925734e-01, 0.00000000e+00, 1.71382217e-05, 5.71274055e-05],\n [0.00000000e+0...","[array([[9.99933541e-01, 0.00000000e+00, 3.16469717e-06, 6.32939434e-05],\n [0.00000000e+0...","[array([[9.99896481e-01, 0.00000000e+00, 7.84236224e-06, 9.56768194e-05],\n [2.81811201e-0...","[array([[9.99926323e-01, 0.00000000e+00, 1.38144656e-05, 5.98626842e-05],\n [7.36713374e-0..."
3,10112056,/proj/NIRAL/users/siyangj/model_10112056/inference_18100_10112056,1,0.825539,0.124662,0.174211,0.737446,1.562985,0.298872,0.992718,0.794706,0.796743,0.786574,"[array([[9.99997979e-01, 0.00000000e+00, 0.00000000e+00, 2.02120241e-06],\n [1.32871972e-0...","[array([[9.90292070e-01, 7.39651809e-06, 5.56957812e-03, 4.13095535e-03],\n [4.97207352e-0...","[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n [3.50592209e-0...","[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n [5.01632815e-0...","[array([[9.99969532e-01, 0.00000000e+00, 9.52123426e-06, 2.09467154e-05],\n [0.00000000e+0...","[array([[9.99984177e-01, 0.00000000e+00, 0.00000000e+00, 1.58234859e-05],\n [0.00000000e+0...","[array([[9.99990589e-01, 0.00000000e+00, 0.00000000e+00, 9.41083469e-06],\n [0.00000000e+0...","[array([[9.99930928e-01, 1.53494062e-06, 6.13976248e-06, 6.13976248e-05],\n [0.00000000e+0..."
4,10112056,/proj/NIRAL/users/siyangj/model_10112056/inference_16500_10112056,1,0.793855,0.189398,0.090426,0.849576,1.643431,0.279823,0.990101,0.825034

In [30]:
inference_eval('10210132','/proj/NIRAL/users/siyangj/model_10210132/inference_60000',
               '/proj/NIRAL/users/siyangj/myData/min_normal/',eval_df,'1')

In [31]:
eval_df_sort = eval_df.sort_values('test_11+22',ascending=False).iloc[:,0:13]

In [34]:
eval_df_sort

,model,inf_dir,test_im,test_11,test_12,test_21,test_22,test_11+22,test_12+21,Dice0,Dice1,Dice2,Dice3
14,10112055.0,/proj/NIRAL/users/siyangj/model_10112055/inference_21900_normal,1,0.861763,0.124861,0.121004,0.822459,1.684223,0.245865,0.991524,0.843291,0.846411,0.833207
12,10112055.0,/proj/NIRAL/users/siyangj/model_10112055/inference_21900_10112055,1,0.861016,0.125160,0.120896,0.821308,1.682324,0.246056,0.995358,0.843021,0.846834,0.845988
35,10210050.0,/proj/NIRAL/users/siyangj/model_10210050/inference_30000,1,0.850768,0.138990,0.139546,0.831239,1.682007,0.278536,0.992319,0.825083,0.831247,0.809081
61,10210130.0,/proj/NIRAL/users/siyangj/model_10210130/inference_55000,1,0.788889,0.194381,0.081139,0.883951,1.672840,0.275520,0.994809,0.827094,0.851354,0.843518
72,10210132.0,/proj/NIRAL/users/siyangj/model_10210132/inference_70000,1,0.861237,0.128304,0.149105,0.802642,1.663880,0.277410,0.995638,0.825284,0.824976,0.832259
58,10210054.0,/proj/NIRAL/users/siyangj/model_10210054/inference_50000,1,0.829719,0.155244,0.121035,0.832751,1.662470,0.276280,0.995374,0.823699,0.834863,0.831108
73,10210132,/proj/NIRAL/users/siyangj/model_10210132/inference_60000,1,0.840177,0.136249,0.108549,0.813160,1.653337,0.244798,0.991953,0.839596,0.841971,0.827433
55,10210050.0,/proj/NIRAL/users/siyangj/model_10210050/inference_60000,1,0.854372,0.132274,0.148030,0.797729,1.652101,0.280304,0.995481,0.820110,0.825959,0.833727
56,10210052.0,/proj/NIRAL/users/siyangj/model_10210052/inference_55400,1,0.862876,0.127103,0.177458,0.788410,1.651286,0.304561,0.995355,0.807689,0.810778,0.819124
54,10210050.0,/proj/NIRAL/users/siyangj/model_10210050/inference_47700,1,0.787571,0.184371,0.088806,0.862547,1.650118,0.273177,0.992172,0.821132,0.846978,0.825889


In [7]:
eval_df_sort.drop([14,12,1,4,5,13],inplace=True)

In [8]:
eval_df_sort

,model,inf_dir,test_im,test_11,test_12,test_21,test_22,test_11+22,test_12+21,Dice0,Dice1,Dice2,Dice3
35,10210050,/proj/NIRAL/users/siyangj/model_10210050/inference_30000,1,0.850768,0.138990,0.139546,0.831239,1.682007,0.278536,0.992319,0.825083,0.831247,0.809081
61,10210130,/proj/NIRAL/users/siyangj/model_10210130/inference_55000,1,0.788889,0.194381,0.081139,0.883951,1.672840,0.275520,0.994809,0.827094,0.851354,0.843518
58,10210054,/proj/NIRAL/users/siyangj/model_10210054/inference_50000,1,0.829719,0.155244,0.121035,0.832751,1.662470,0.276280,0.995374,0.823699,0.834863,0.831108
55,10210050,/proj/NIRAL/users/siyangj/model_10210050/inference_60000,1,0.854372,0.132274,0.148030,0.797729,1.652101,0.280304,0.995481,0.820110,0.825959,0.833727
56,10210052,/proj/NIRAL/users/siyangj/model_10210052/inference_55400,1,0.862876,0.127103,0.177458,0.788410,1.651286,0.304561,0.995355,0.807689,0.810778,0.819124
54,10210050,/proj/NIRAL/users/siyangj/model_10210050/inference_47700,1,0.787571,0.184371,0.088806,0.862547,1.650118,0.273177,0.992172,0.821132,0.846978,0.825889
70,10210132,/proj/NIRAL/users/siyangj/model_10210132/inference_55000,1,0.788230,0.190931,0.072818,0.861572,1.649802,0.263749,0.995917,0.831604,0.848871,0.843631
68,10210054,/proj/NIRAL/users/siyangj/model_10210054/inference_57300,1,0.794176,0.185833,0.093085,0.852272,1.646448,0.278917,0.995408,0.821614,0.839846,0.833980
63,10210132,/proj/NIRAL/users/siyangj/model_10210132/inference_45000,1,0.798356,0.177805,0.075448,0.843776,1.642132,0.253253,0.994820,0.835899,0.845961,0.837082
62,10210130,/proj/NIRAL/users/siyangj/model_10210130/inference_45000,1,0.840122,0.147062,0.156935,0.799764,1.639886,0.303997,0.991060,0.809171,0.816202,0.808539


In [9]:
## Majority vote for the top ones

for top_num in [1,3,5,7,9,11,13,15]:
    im_arr = np.zeros([96,112,96,top_num])
    for i in range(top_num):
        im_arr[:,:,:,i] = sitk.GetArrayFromImage(
                sitk.ReadImage(
                    os.path.join(eval_df_sort.iloc[i,1],'1_niftynet_out.nii.gz')))

    true_arr = sitk.GetArrayFromImage(
        sitk.ReadImage('/proj/NIRAL/users/siyangj/myData/min_normal/seg_inf_1/1.nrrd'))

    label_vote_arr = np.zeros([96,112,96,4])
    for i in range(4):
        label_vote_arr[:,:,:,i] = (im_arr==i).sum(axis=3)

    res_arr = label_vote_arr.argmax(axis=3)

    res_arr.shape

    c_mat = skl.metrics.confusion_matrix(true_arr.flatten(),res_arr.flatten())
    p_mat = c_mat.T / c_mat.astype(np.float).sum(axis=1)
    p_mat = p_mat.T
    print('top_num = {:d}'.format(top_num))
    print(p_mat)
    print(Dice(res_arr,true_arr))

top_num = 1
[[9.99249704e-01 0.00000000e+00 2.85438750e-04 4.64857393e-04]
 [9.68858131e-04 8.50768166e-01 1.38989619e-01 9.27335640e-03]
 [2.47586994e-03 1.39546219e-01 8.31238515e-01 2.67393953e-02]
 [6.24177624e-02 2.07506896e-02 1.79085566e-01 7.37745982e-01]]
[0.99231927 0.82508269 0.83124655 0.80908148]
top_num = 3
[[9.99133490e-01 0.00000000e+00 1.38641679e-04 7.27868812e-04]
 [7.91695502e-04 8.28844291e-01 1.59031142e-01 1.13328720e-02]
 [2.41784174e-03 1.07479835e-01 8.59769048e-01 3.03332753e-02]
 [4.26027544e-02 1.65498689e-02 1.48646673e-01 7.92200704e-01]]
[0.99435505 0.83214754 0.84699829 0.83739691]
top_num = 5
[[9.98939799e-01 0.00000000e+00 1.73302098e-04 8.86898973e-04]
 [1.77162630e-04 8.45115571e-01 1.44348789e-01 1.03584775e-02]
 [1.66734366e-03 1.23236426e-01 8.41722277e-01 3.33739531e-02]
 [3.52147682e-02 2.22419322e-02 1.41921461e-01 8.00621838e-01]]
[0.99533165 0.83094183 0.84203988 0.83958605]
top_num = 7
[[9.98968343e-01 0.00000000e+00 1.54952464e-04 8.767047

In [19]:
arr1 = sitk.GetArrayFromImage(
        sitk.ReadImage('/proj/NIRAL/users/siyangj/myData/min_normal/seg_inf_1/1.nrrd'))
arr2 = sitk.GetArrayFromImage(
        sitk.ReadImage('/proj/NIRAL/users/siyangj/myData/min_normal/seg/1.nrrd'))

In [21]:
(arr1 != arr2).sum()

4282

In [24]:
4282/(96*96*112)

0.004148453000992063